<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Planet
[Github Repository](https://github.com/progirep/planet)

In [ ]:
%%capture

# Clone the repo
!git clone https://github.com/progirep/planet.git

In [ ]:
%%capture
# we need a script to convert the neural network description in to the "prototxt" format (produced by Caffe) into a json file
!cd tools
!wget https://gist.github.com/progirep/fd7d2dc120862faa984a70f503611013/raw/260e1e76cebd0ea58bf1a03b64c3f1e0002fc677/csv_to_hdf5_supervised_classification.py 

# we need a second script to generate a database in "HDF5" format from comma-separated value files
!wget https://raw.githubusercontent.com/vadimkantorov/caffemodel2json/3a8fd443bf1596dad5f517aecdef08a81bf73bfe/caffemodel2json.py

In [ ]:
%%capture

# install packages in order to build PLANET
!sudo apt-get install libglpk-dev
!sudo apt-get install qt5-qmake
!sudo apt-get install valgrind
!sudo apt-get install libltdl-dev
!sudo apt-get install protobuf-compiler

In [ ]:
%%capture

# compile the source code
%cd planet/src
%ls
!qmake Tool.pro
!make

In [ ]:
%%capture

# install Caffe and Yices
# change root path of #CAFFE and #YICES
!sudo apt install caffe-cuda
!sudo add-apt-repository ppa:sri-csl/formal-methods -qq
!sudo apt-get update
!sudo apt-get install yices2

In [ ]:
# envs probably useless
%env YICES=/usr
%env CAFFE=/usr
%cd /content/planet/casestudies/MNIST/

In [ ]:
%%bash

export YICES=/usr
export CAFFE=/sr
set -e
caffe train --solver=lenet_solver.prototxt $@ 

Current Problem:
https://github.com/BVLC/caffe/issues/2780
> Check failed: mdb_status == 0



#Current Problems
* csv_to_hdf5_supervised_classificatoin.py doesn't work, because it's Python2
* Error with caffe train - not sure why -https://github.com/BVLC/caffe/issues/2780

#TODO's
* Try to do a local jupyter notebook instead of colab
* Continue the "Running the Collision Avoidance Benchmarks" Section

